In [1]:
import pandas as pd
job_id='dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-sc2bx1af4k'
df = pd.read_csv(f'../../extractions/{job_id}/result.csv')
df

,source_name,dateline,ingestion_datetime,currency_codes,company_codes_association_ticker_exchange,title,snippet,company_codes_lineage_ticker_exchange,an,company_codes_occur_ticker_exchange,...,document_type,modification_datetime,company_codes,action,region_codes,market_index_codes,company_codes_about_ticker_exchange,company_codes_occur,section,company_codes_relevance_ticker_exchange
0,The Telegraph,NaN,1742603443000,NaN,NaN,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",NaN,ABPTEL0020250322el3m00021,",ALIZY:PINX,ALV:XSWX,ALVE:XFRA,ALV:XBSE,1ALV:X...",...,article,1742603443000,",allanz,allanz,allanz,bajaal,bajaal,blfima,blf...",add,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",NaN,",ALIZY:PINX,ALV:XSWX,ALVE:XFRA,ALV:XBSE,1ALV:X...",",zozfrr,srg,pru,piasmc,paions,icplic,hcltec,bl...",NaN,",ALIZY:PINX,ALV:XSWX,ALVE:XFRA,ALV:XBSE,1ALV:X..."
1,AfU Company Information: Mutual Fund Holdings ...,NaN,1739990205000,NaN,NaN,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,NaN,ACIIBC0020250219el2j0008o,NaN,...,article,1739990205000,",ffi,ffi,ffi,metzam,metzam,univin,univin,",add,",eecz,eurz,gfr,weurz,",NaN,NaN,",univin,metzam,ffi,",NaN,NaN
2,AfU Company Information: Mutual Fund Holdings ...,NaN,1742409206000,NaN,NaN,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,NaN,ACIIBC0020250319el3j0001j,NaN,...,article,1742409206000,",ffi,ffi,ffi,tburk,tburk,univin,univin,",add,",eecz,eurz,gfr,weurz,",NaN,NaN,",univin,tburk,ffi,",NaN,NaN
3,AfU Company Information: Mutual Fund Holdings ...,NaN,1736336156000,NaN,NaN,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,NaN,ACIIBC0020250108el180005a,NaN,...,article,1736336156000,",ffi,ffi,ffi,metzam,metzam,univin,univin,",add,",eecz,eurz,gfr,weurz,",NaN,NaN,",univin,metzam,ffi,",NaN,NaN
4,AfU Company Information: Mutual Fund Holdings ...,NaN,1735133756000,NaN,NaN,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,NaN,ACIIBC0020241225ekcp0008o,NaN,...,article,1735133756000,",ffi,ffi,ffi,metzam,metzam,univin,univin,",add,",eecz,eurz,gfr,weurz,",NaN,NaN,",univin,metzam,ffi,",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46672,walesonline.co.uk,NaN,1743066934000,NaN,NaN,"'You're not on your own, and everyone makes mi...",The NatWest Entrepreneur Accelerator programme...,NaN,WALESONL20250327el3r0012y,",S146N:XAMS,RYSD:XETR,AGKSZX:XASX,AA2H5C:XFRA,...",...,article,1743066934000,",rbsct,rbsct,rbsct,",add,",cardi,eurz,uk,wales,weurz,",NaN,",S146N:XAMS,RYSD:XETR,AGKSZX:XASX,AA2H5C:XFRA,...",",rbsct,",Partner Stories,",S146N:XAMS,RYSD:XETR,AGKSZX:XASX,AA2H5C:XFRA,..."
46673,walesonline.co.uk,NaN,1741356329000,NaN,NaN,Boots takeover completed amid uncertainty on i...,Walgreens Boots Alliance said Sycamore will pa...,NaN,WALESONL20250307el37002mh,NaN,...,article,1741356329000,",bootcl,bootz,gmyvve,rteaid,sycpau,sycpau,sycp...",add,",eurz,namz,uk,usa,usc,usil,weurz,",",xf500,",NaN,",wlgrn,sycpau,",UK News,",RTA1:XFRA,RAD:XNYS,RTA1:XSTU,RTA1:XFRA,RADCQ:..."
46674,walesonline.co.uk,NaN,1734975943000,NaN,NaN,DWP update as plans to merge two major benefit...,The Department for Work and Pensions (DWP) has...,NaN,WALESONL20241223ekcn00439,NaN,...,article,1734975943000,",govgb,govgb,ibm,ibm,ukdwp,ukdwp,ukdwp,",add,",eurz,uk,weurz,",NaN,NaN,",ukdwp,ibm,govgb,",UK News,NaN
46675,walesonline.co.uk,NaN,1744045242000,NaN,NaN,Financial services firm that collapsed owing c...,Neal Griffith was chief executive of Optima Wo...,NaN,WALESONL20250407el47004ed,NaN,...,article,1744045242000,",fisgbo,fisgbo,fisgbo,fsrcsl,fsrcsl,moilga,omb...",add,",cardi,eurz,uk,wales,weurz,",NaN,NaN,",vibidl,vhskrn,ombud,fsrcsl,fisgbo,",Rugby News,NaN


## Initial preprocessing

#### Dropping irrelevant cols

In [2]:
cols2drop=['dateline','currency_codes','availability_datetime', 'copyright', 'language_code','region_of_origin','byline','person_codes','publication_date','publisher_name','credit','art','document_type','modification_datetime','market_index_codes'] + [c for c in df.columns if c.endswith('_exchange')]
df = df.drop(columns=cols2drop, errors='ignore') 

In [3]:
## Converting dt
for dt_column in ["modification_date", 'modification_datetime',"ingestion_datetime", "publication_datetime","publication_date"]:
    if dt_column in df.columns:
        df[dt_column] = pd.to_datetime(df[dt_column], unit='ms')

## Aggregating different company codes columns
cc_columns = sorted([c for c in df.columns if c.startswith('company_codes')])
for col in cc_columns:
    df[col] = df[col].apply(lambda x: ','.join(list(set(x.lstrip(',').split(','))-{'',})) if isinstance(x, str) else x)
    df[col] = df[col].fillna("")#.apply(list)
df['all_company_codes'] = df[cc_columns].agg(lambda x: ','.join(x), axis=1).apply(lambda x: ','.join(sorted(list(set(x.split(','))-{''}))))

df = df.drop(columns=cc_columns, errors='ignore') 
df = df.rename(columns={'all_company_codes':'company_codes'})

### Ordering cols logically

In [4]:
columns = ['source_name','title','snippet','body','section','word_count','source_code','industry_codes','company_codes','subject_codes','publication_datetime','ingestion_datetime','modification_date','region_codes','an','action']
assert all([c in df.columns for c in columns]), f"Missing columns: {[c for c in columns if c not in df.columns]}"
df = df[columns]
df

,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action
0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",",c13,c18,c184,cactio,ccat,cpartn,ncat,nfact,nf...",2025-03-22 00:00:00,2025-03-22 00:30:43,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add
1,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-02-19 00:00:00,2025-02-19 18:36:45,2025-02-20 07:29:39.241,",eecz,eurz,gfr,weurz,",ACIIBC0020250219el2j0008o,add
2,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,526,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,tburk,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-03-19 00:00:00,2025-03-19 18:33:26,2025-03-20 07:29:40.635,",eecz,eurz,gfr,weurz,",ACIIBC0020250319el3j0001j,add
3,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-01-08 00:00:00,2025-01-08 11:35:56,2025-01-09 07:18:20.940,",eecz,eurz,gfr,weurz,",ACIIBC0020250108el180005a,add
4,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2024-12-25 00:00:00,2024-12-25 13:35:56,2024-12-26 07:13:02.781,",eecz,eurz,gfr,weurz,",ACIIBC0020241225ekcp0008o,add
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46672,walesonline.co.uk,"'You're not on your own, and everyone makes mi...",The NatWest Entrepreneur Accelerator programme...,"However, as an entrepreneur, NatWest Cymru is ...",Partner Stories,935,WALESONL,",i3302,i814,i81402,ibnk,icomp,icph,ifinal,itech,",rbsct,",c11,c24,ccat,cprdop,nadvtr,ncat,",2025-03-27 09:00:00,2025-03-27 09:15:34,2025-03-28 07:37:05.166,",cardi,eurz,uk,wales,weurz,",WALESONL20250327el3r0012y,add
46673,walesonline.co.uk,Boots takeover completed amid uncertainty on i...,Walgreens Boots Alliance said Sycamore will pa...,Shareholders could potentially receive up to a...,UK News,566,WALESONL,",i64,i643,i654,i81502,i8150203,ialtinv,ifinal,...","bootcl,bootz,gmyvve,rteaid,sycpau,wlgrn",",c18,c181,cacqu,cactio,ccat,ncat,nfact,nfce,nf...",2025-03-07 13:38:23,2025-03-07 14:05:29,2025-03-08 07:18:08.673,",eurz,namz,uk,usa,usc,usil,weurz,",WALESONL20250307el37002mh,add
46674,walesonline.co.uk,DWP update as plans to merge two major benefit...,The Department for Work and Pensions (DWP) has...,Initially introduced by the Conservative gover...,UK News,539,WALESONL,",i81502,ifinal,iinv,ipension,ippf,","govgb,ibm,ukdwp",",c18,c181,cacqu,cactio,ccat,cmerg,ncat,nfact,n...",2024-12-23 17:23:17,2024-12-23 17:45:43,2024-12-24 07:24:48.622,",eurz,uk,weurz,",WALESONL20241223ekcn00439,add
46675,walesonline.co.uk,Financial services firm that collapsed owing c...,Neal Griffith was chief executive of Optima Wo...,Mr Griffith is the former chief executive and ...,Rugby News,1210,WALESONL,",i836,iacc,ibcs,ifinal,","fisgbo,fsrcsl,moilga,ombud,vh

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46677 entries, 0 to 46676
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   source_name           46677 non-null  object        
 1   title                 46677 non-null  object        
 2   snippet               44489 non-null  object        
 3   body                  46677 non-null  object        
 4   section               1697 non-null   object        
 5   word_count            46677 non-null  int64         
 6   source_code           46677 non-null  object        
 7   industry_codes        46677 non-null  object        
 8   company_codes         46677 non-null  object        
 9   subject_codes         46677 non-null  object        
 10  publication_datetime  46677 non-null  datetime64[ns]
 11  ingestion_datetime    46677 non-null  datetime64[ns]
 12  modification_date     46677 non-null  datetime64[ns]
 13  region_codes    

### Decoding company names

In [6]:
from factiva.news.taxonomy import Taxonomy
user_key = 'LKbI9fY6ZEPU8RcJUxQHjwKbld52WGt0'
taxonomy = Taxonomy(user_key)
company_codes_df = taxonomy.get_category_codes('companies')
company_codes_df = company_codes_df[~company_codes_df.index.duplicated(keep='first')]
company_codes_df

,description,ticker,ticker_exchange,exchange,primary_exchange
code,,,,,
CAVRDJ,Casa & Video Brasil Sa,NaN,NaN,NaN,NaN
DAUMRC,Developmental Association Uralskaya Mramornaya...,NaN,NaN,NaN,NaN
BRTRAL,Brandt Tractor Ltd,NaN,NaN,NaN,NaN
HPMMCM,Hoa Phat Mechanism Manufacturing Co. Ltd,NaN,NaN,NaN,NaN
NGFNNE,Ningxia Guofan New Energy Co. Ltd.,NaN,NaN,NaN,NaN
...,...,...,...,...,...
SFNDHL,Samadhan Finserve Nidhi Limited,NaN,NaN,NaN,NaN
CJIRCB,Changchun Jingkai Rongfeng Country Bank Co. Ltd.,NaN,NaN,NaN,NaN
KAVPAF,Kavita Uab,NaN,NaN,NaN,NaN


In [7]:
company_dict = company_codes_df.description.to_dict()
def extract_company_names(company_codes_list):
    company_codes_list = company_codes_list.upper().split(',')
    
    company_names = []
    for code in company_codes_list:
        if code in company_dict.keys():
            company_name = company_dict.get(code,code)
            company_names.append(company_name)
        else:
            company_names.append(code)
    return ','.join(company_names)

df['company_names'] = df['company_codes'].apply(lambda x: extract_company_names(x))
df

,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action,company_names
0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",",c13,c18,c184,cactio,ccat,cpartn,ncat,nfact,nf...",2025-03-22 00:00:00,2025-03-22 00:30:43,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL..."
1,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-02-19 00:00:00,2025-02-19 18:36:45,2025-02-20 07:29:39.241,",eecz,eurz,gfr,weurz,",ACIIBC0020250219el2j0008o,add,"3i Group PLC,Metzler Asset Management GmbH,Uni..."
2,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,526,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,tburk,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-03-19 00:00:00,2025-03-19 18:33:26,2025-03-20 07:29:40.635,",eecz,eurz,gfr,weurz,",ACIIBC0020250319el3j0001j,add,"3i Group PLC,HSBC Trinkaus und Burkhardt AG,Un..."
3,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-01-08 00:00:00,2025-01-08 11:35:56,2025-01-09 07:18:20.940,",eecz,eurz,gfr,weurz,",ACIIBC0020250108el180005a,add,"3i Group PLC,Metzler Asset Management GmbH,Uni..."
4,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2024-12-25 00:00:00,2024-12-25 13:35:56,2024-12-26 07:13:02.781,",eecz,eurz,gfr,weurz,",ACIIBC0020241225ekcp0008o,add,"3i Group PLC,Metzler Asset Management GmbH,Uni..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46672,walesonline.co.uk,"'You're not on your own, and everyone makes mi...",The NatWest Entrepreneur Accelerator programme...,"However, as an entrepreneur, NatWest Cymru is ...",Partner Stories,935,WALESONL,",i3302,i814,i81402,ibnk,icomp,icph,ifinal,itech,",rbsct,",c11,c24,ccat,cprdop,nadvtr,ncat,",2025-03-27 09:00:00,2025-03-27 09:15:34,2025-03-28 07:37:05.166,",cardi,eurz,uk,wales,weurz,",WALESONL20250327el3r0012y,add,NatWest Group PLC
46673,walesonline.co.uk,Boots takeover completed amid uncertainty on i...,Walgreens Boots Alliance said Sycamore will pa...,Shareholders could potentially receive up to a...,UK News,566,WALESONL,",i64,i643,i654,i81502,i8150203,ialtinv,ifinal,...","bootcl,bootz,gmyvve,rteaid,sycpau,wlgrn",",c18,c181,cacqu,cactio,ccat,ncat,nfact,nfce,nf...",2025-03-07 13:38:23,2025-03-07 14:05:29,2025-03-08 07:18:08.673,",eurz,namz,uk,usa,usc,usil,weurz,",WALESONL20250307el37002mh,add,"Boots UK Limited,Alliance Boots GmbH,VillageMD..."
46674,walesonline.co.uk,DWP update as plans to merge two major benefit...,The Department for Work and Pensions (DWP) has...,Initially introduced by the Conservative gover...,UK News,539,WALESONL,",i81502,ifinal,iinv,ipension,ippf,","govgb,ibm,ukdwp",",c18,c181,cacqu,cactio,ccat,cmerg,ncat,nfact,n...",2024-12-23 17:23:17

### Decoding industries

In [8]:
industries_df = taxonomy.get_category_codes('industries')
industries_mapping_dict = industries_df.description.to_dict()

def extract_industries(industry_codes_list):
    industry_codes_list = sorted(list(set(industry_codes_list.split(','))-{''}))
    
    industry_descriptions = []
    for code in industry_codes_list:
        if code in industries_mapping_dict.keys():
            industry_desc = industries_mapping_dict.get(code,code)
            industry_descriptions.append(industry_desc)
        else:
            industry_descriptions.append(code)
    return ','.join(industry_descriptions)
df['industries'] = df['industry_codes'].apply(lambda x: extract_industries(x))
df

,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action,company_names,industries
0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",",c13,c18,c184,cactio,ccat,cpartn,ncat,nfact,nf...",2025-03-22 00:00:00,2025-03-22 00:30:43,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F..."
1,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-02-19 00:00:00,2025-02-19 18:36:45,2025-02-20 07:29:39.241,",eecz,eurz,gfr,weurz,",ACIIBC0020250219el2j0008o,add,"3i Group PLC,Metzler Asset Management GmbH,Uni...","Trusts/Funds/Financial Vehicles,Private Equity..."
2,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,526,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,tburk,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-03-19 00:00:00,2025-03-19 18:33:26,2025-03-20 07:29:40.635,",eecz,eurz,gfr,weurz,",ACIIBC0020250319el3j0001j,add,"3i Group PLC,HSBC Trinkaus und Burkhardt AG,Un...","Trusts/Funds/Financial Vehicles,Private Equity..."
3,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-01-08 00:00:00,2025-01-08 11:35:56,2025-01-09 07:18:20.940,",eecz,eurz,gfr,weurz,",ACIIBC0020250108el180005a,add,"3i Group PLC,Metzler Asset Management GmbH,Uni...","Trusts/Funds/Financial Vehicles,Private Equity..."
4,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2024-12-25 00:00:00,2024-12-25 13:35:56,2024-12-26 07:13:02.781,",eecz,eurz,gfr,weurz,",ACIIBC0020241225ekcp0008o,add,"3i Group PLC,Metzler Asset Management GmbH,Uni...","Trusts/Funds/Financial Vehicles,Private Equity..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46672,walesonline.co.uk,"'You're not on your own, and everyone makes mi...",The NatWest Entrepreneur Accelerator programme...,"However, as an entrepreneur, NatWest Cymru is ...",Partner Stories,935,WALESONL,",i3302,i814,i81402,ibnk,icomp,icph,ifinal,itech,",rbsct,",c11,c24,ccat,cprdop,nadvtr,ncat,",2025-03-27 09:00:00,2025-03-27 09:15:34,2025-03-28 07:37:05.166,",cardi,eurz,uk,wales,weurz,",WALESONL20250327el3r0012y,add,NatWest Group PLC,"Computers/Consumer Electronics,Banking,Commerc..."
46673,walesonline.co.uk,Boots takeover completed amid uncertainty on i...,Walgreens Boots Alliance said Sycamore will pa...,Shareholders could potentially receive up to a...,UK News,566,WALESONL,",i64,i643,i654,i81502,i8150203,ialtinv,ifinal,...","bootcl,bootz,gmyvve,rteaid,sycpau,wlgrn",",c18,c181,cacqu,cactio,ccat,ncat,nfact,nfce,nf...",2025-03-07 13:38:23,2025-03-07 14:05:29,2025-03-08 07:18:08.673,",eurz,namz,uk,usa,usc,usil,weurz,",WALESONL20250307el37002mh,add,"Boots UK Limited,Alliance Boots GmbH,VillageM

## Splitting long articles
* Given volume and potential hardware limitations/speed considerations - TextTilingTokenizer seems [a good choice](https://medium.com/blue-orange-digital/document-chunking-guided-journey-a-greedy-start-e39a1dff15c3)

In [9]:
df

,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action,company_names,industries
0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",",c13,c18,c184,cactio,ccat,cpartn,ncat,nfact,nf...",2025-03-22 00:00:00,2025-03-22 00:30:43,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F..."
1,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-02-19 00:00:00,2025-02-19 18:36:45,2025-02-20 07:29:39.241,",eecz,eurz,gfr,weurz,",ACIIBC0020250219el2j0008o,add,"3i Group PLC,Metzler Asset Management GmbH,Uni...","Trusts/Funds/Financial Vehicles,Private Equity..."
2,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,526,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,tburk,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-03-19 00:00:00,2025-03-19 18:33:26,2025-03-20 07:29:40.635,",eecz,eurz,gfr,weurz,",ACIIBC0020250319el3j0001j,add,"3i Group PLC,HSBC Trinkaus und Burkhardt AG,Un...","Trusts/Funds/Financial Vehicles,Private Equity..."
3,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2025-01-08 00:00:00,2025-01-08 11:35:56,2025-01-09 07:18:20.940,",eecz,eurz,gfr,weurz,",ACIIBC0020250108el180005a,add,"3i Group PLC,Metzler Asset Management GmbH,Uni...","Trusts/Funds/Financial Vehicles,Private Equity..."
4,AfU Company Information: Mutual Fund Holdings ...,"Agentur für Unternehmensnachrichten GmbH, mutu...",company information\n\ncompany 3i G...,all german mutual funds\n\nshare_name ...,NaN,487,ACIIBC,",i81502,i8150203,i8150211,ialtinv,ifinal,iinv,","ffi,metzam,univin",",c18,c181,cactio,ccat,ncat,nfact,nfcpin,",2024-12-25 00:00:00,2024-12-25 13:35:56,2024-12-26 07:13:02.781,",eecz,eurz,gfr,weurz,",ACIIBC0020241225ekcp0008o,add,"3i Group PLC,Metzler Asset Management GmbH,Uni...","Trusts/Funds/Financial Vehicles,Private Equity..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46672,walesonline.co.uk,"'You're not on your own, and everyone makes mi...",The NatWest Entrepreneur Accelerator programme...,"However, as an entrepreneur, NatWest Cymru is ...",Partner Stories,935,WALESONL,",i3302,i814,i81402,ibnk,icomp,icph,ifinal,itech,",rbsct,",c11,c24,ccat,cprdop,nadvtr,ncat,",2025-03-27 09:00:00,2025-03-27 09:15:34,2025-03-28 07:37:05.166,",cardi,eurz,uk,wales,weurz,",WALESONL20250327el3r0012y,add,NatWest Group PLC,"Computers/Consumer Electronics,Banking,Commerc..."
46673,walesonline.co.uk,Boots takeover completed amid uncertainty on i...,Walgreens Boots Alliance said Sycamore will pa...,Shareholders could potentially receive up to a...,UK News,566,WALESONL,",i64,i643,i654,i81502,i8150203,ialtinv,ifinal,...","bootcl,bootz,gmyvve,rteaid,sycpau,wlgrn",",c18,c181,cacqu,cactio,ccat,ncat,nfact,nfce,nf...",2025-03-07 13:38:23,2025-03-07 14:05:29,2025-03-08 07:18:08.673,",eurz,namz,uk,usa,usc,usil,weurz,",WALESONL20250307el37002mh,add,"Boots UK Limited,Alliance Boots GmbH,VillageM

In [10]:
## requirements: tqdm-joblib, nltk, 
import re, os

from functools import partial
from multiprocessing import Value

from bs4 import BeautifulSoup
from nltk.tokenize import TextTilingTokenizer
from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib 
from tqdm.autonotebook import tqdm
tqdm.pandas()

TOKEN_RX = re.compile(r"[A-Za-z0-9]+(?:'[A-Za-z0-9]+)?")

def factiva_wordcount(html: str) -> int:
    """
    Approximate Factiva 'WC' very closely (±0–2 % on real wires).
    1. Strip markup quickly with lxml‑backed BeautifulSoup.
    2. Replace hyphens & slashes with space so they split tokens.
    3. Count regex‑defined tokens.
    """
    # -------- strip HTML completely --------
    txt = BeautifulSoup(html, "lxml").get_text(" ")

    # -------- normalise separators ---------
    txt = txt.replace("‑", "-")           # unicode hyphen
    txt = re.sub(r"[/-]", " ", txt)       # split hyphen/slash compounds
    txt = txt.replace("\u00A0", " ")      # non‑breaking sp

    # -------- tokenise & count ------------
    return len(TOKEN_RX.findall(txt))



bodies  = df["body"].tolist() 
word_counts  = df["word_count"].tolist()   
CPUS    = os.cpu_count()
ERRORS       = Value("i", 0)                  # shared int (32-bit)

tokenizer = TextTilingTokenizer()


def _tile(doc, wc, wc_min=100):
    if wc < wc_min:
        return [doc], 0           # segments, errflag

    try:
        return tokenizer.tokenize(doc), 0
    except Exception as e:
        return [doc], 1

with tqdm_joblib(tqdm(total=len(bodies), desc="Tiling")):
    results = Parallel(n_jobs=CPUS, backend="multiprocessing", batch_size=4)(
        delayed(_tile)(doc, wc) for doc, wc in zip(bodies, word_counts)
    )
segments, err_flags = zip(*results)

df["segments"]      = segments
df['num_segments'] = df['segments'].apply(lambda x: len(x))
error_count         = sum(err_flags)

print(f"{error_count} documents raised an error during tiling.")
df_exploded = df.explode('segments')
df_exploded


/home/alk/Projects/carveouts_poc/venv/lib/python3.12/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Tiling:   0%|          | 0/46677 [00:00<?, ?it/s]

  0%|          | 0/46677 [00:00<?, ?it/s]

16851 documents raised an error during tiling.


,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action,company_names,industries,segments,num_segments
0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",",c13,c18,c184,cactio,ccat,cpartn,ncat,nfact,nf...",2025-03-22 00:00:00,2025-03-22 00:30:43,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",London-based multinational insurance and asset...,5
0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",",c13,c18,c184,cactio,ccat,cpartn,ncat,nfact,nf...",2025-03-22 00:00:00,2025-03-22 00:30:43,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",\n\n“Subject to obtaining regulatory approvals...,5
0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",",c13,c18,c184,cactio,ccat,cpartn,ncat,nfact,nf...",2025-03-22 00:00:00,2025-03-22 00:30:43,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...","\n\nMeanwhile, Bloomberg has reported that bil...",5
0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",",c13,c18,c184,cactio,ccat,cpartn,ncat,nfact,nf...",2025-03-22 00:00:00,2025-03-22 00:30:43,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",\n\nWith global reinsurance major Swiss RE pro...,5
0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",",c13,c18,c184,cactio,ccat,cpartn,ncat,nfact,nf...",2025-03-22 00:00:00,2025-03-22 00:30:43,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",\n\nWith life insurance penetration at a level...,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46676,walesonline.co.uk,One of Cardiff's biggest employers is moving t...,Car financing firm MotoNovo is moving its HQ f...,"It is vacating around 70,000 sq ft of space at...",Latest Wales News,717,WALESONL,",i814,i81402,i81404,i81501,i8150104,i831,ibnk,...","acacq,bredo,ckark,crdffu,dunbst,ltgil,momlas,p...",",c02,ccat,chqr,coffs,",2025-02-12 12:55:37,2025-02-12 13:21:38,2025-02-13 07:39:35.418,",cardi,eurz,uk,wales,weurz,",WALESONL20250212el2c002pa,add,"Aldermore Group PLC,Brewin Dolphin Holdings PL...","Banking,Commercial Banking,Digital Banking,Cre...",\n\nAfter subletting space 

### Hashing and dedup
* How to drop duplicates without over-relying on Transformers? -> all sorts of hashing


In [11]:
## requirements: datasketch, simhash
import hashlib

from collections import defaultdict
from typing import Dict, Iterable, List, Set

from datasketch import MinHash, MinHashLSH
from simhash import Simhash, SimhashIndex


re_ws  = re.compile(r"\s+")

df_exploded["segment_norm"] = (
    df_exploded["segments"]
      .str.lower()
      .str.replace(re_ws, " ", regex=True)   # regex compiled once in C
      .str.strip()
)

# 2 — hash (still a Python loop but the strings are now short & tidy)
df_exploded["segment_hash"] = df_exploded["segment_norm"].map(
        lambda t: hashlib.md5(t.encode()).hexdigest()
)
print(df_exploded['segment_hash'].nunique(), df_exploded['segments'].nunique(), df_exploded['segments'].nunique()/df_exploded['segment_hash'].nunique())
df_exploded.drop_duplicates(subset=['segment_hash'], inplace=True)
df_exploded.reset_index(inplace=True)
df_exploded

213364 215284 1.0089987064359498


,index,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,...,modification_date,region_codes,an,action,company_names,industries,segments,num_segments,segment_norm,segment_hash
0,0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",...,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",London-based multinational insurance and asset...,5,london-based multinational insurance and asset...,a176a4521fa13765d1ec53bf0557756c
1,0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",...,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",\n\n“Subject to obtaining regulatory approvals...,5,"“subject to obtaining regulatory approvals, pr...",d3ebc36206e177294b3af34911353648
2,0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",...,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...","\n\nMeanwhile, Bloomberg has reported that bil...",5,"meanwhile, bloomberg has reported that billion...",200f451e94d7320d8bebdde26c5da12d
3,0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",...,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",\n\nWith global reinsurance major Swiss RE pro...,5,with global reinsurance major swiss re project...,93fe39a844b1d00b9eeb9a925f165143
4,0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",...,2025-03-23 07:18:51.972,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",\n\nWith life insurance penetration at a level...,5,with life insurance penetration at a level of ...,c7e0dce171e1e768d3793b239d13bba4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213359,46676,walesonline.co.uk,One of Cardiff's biggest employers is moving t...,Car financing firm MotoNovo is moving its HQ f...,"It is vacating around 70,000 sq ft of space at...",Latest Wales News,717,WALESONL,",i814,i81402,i81404,i81501,i8150104,i831,ibnk,...","acacq,bredo,ckark,crdffu,dunbst,ltgil,momlas,p...",...,2025-02-13 07:39:35.418,",cardi,eurz,uk,wales,weurz,",WALESONL20250212el2c002pa,add,"Aldermore Group PLC,Brewin Dolphin Holdings PL...","Banking,Commercial Banking,Digital Banking,Cre...",\n\nAfter subletting space to financial servic...,6,after subletting space to financial services f...,969

In [12]:
def shingles(text: str, n: int = 8) -> Iterable[str]:
    """Yield n‑word shingles (lower‑cased, non‑word chars collapsed)."""
    tokens = re.sub(r"\W+", " ", text).lower().split()
    for i in range(max(0, len(tokens) - n + 1)):
        yield " ".join(tokens[i : i + n])


def union_find(n: int):
    """Classic union‑find / disjoint‑set with path compression & union‑by‑size."""
    parent = list(range(n))
    size = [1] * n

    def find(x):
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union(a, b):
        ra, rb = find(a), find(b)
        if ra == rb:
            return
        if size[ra] < size[rb]:
            ra, rb = rb, ra
        parent[rb] = ra
        size[ra] += size[rb]

    return parent, find, union

def minhash_dedup(
    df: pd.DataFrame,
    col: str = "segment_norm",
    num_perm: int = 128,
    jaccard_thresh: float = 0.9,
) -> Dict[int, List[int]]:
    """Return clusters {root_idx: [member_idxs …]} by MinHash‑LSH.@≥thresh"""

    lsh = MinHashLSH(threshold=jaccard_thresh, num_perm=num_perm)
    mhs: List[MinHash] = []

    # 0‑based reindex for contiguous union‑find
    df = df.reset_index(drop=True)

    for idx, txt in tqdm(df[col].items(), desc="MinHash: build", unit="doc"):
        mh = MinHash(num_perm=num_perm)
        for sh in shingles(txt, n=8):
            mh.update(sh.encode())
        lsh.insert(str(idx), mh)
        mhs.append(mh)

    parent, find, union = union_find(len(mhs))

    for idx, mh in tqdm(enumerate(mhs), total=len(mhs), desc="MinHash: query", unit="doc"):
        for candidate in lsh.query(mh):
            j = int(candidate)
            if j <= idx:
                continue  # avoid self / duplicate pairs
            union(idx, j)

    clusters: Dict[int, List[int]] = defaultdict(list)
    for i in range(len(mhs)):
        clusters[find(i)].append(i)
    return clusters


def containment_dedup(
    df: pd.DataFrame,
    candidates: Iterable[int],
    col: str = "segment_norm",
) -> Set[int]:
    """Return the indices **to keep** after removing verbatim substrings.

    Strategy: sort survivors by length; for each *longer* doc scan shorter ones.
    Optional optimisation with Aho‑Corasick if pyahocorasick is installed.
    """

    try:
        import ahocorasick

        automaton = ahocorasick.Automaton()
        texts = {i: df.at[i, col] for i in candidates}
        # shorter → pattern, longer → search text
        by_len = sorted(texts.items(), key=lambda kv: len(kv[1]))
        keep: Set[int] = set(texts.keys())
        for i, (idx_i, pat) in enumerate(by_len):
            automaton.add_word(pat, (idx_i, pat))
        automaton.make_automaton()

        for idx_long, txt_long in reversed(by_len):
            for end_idx, (idx_short, _pat) in automaton.iter(txt_long):
                if idx_short == idx_long or idx_short not in keep:
                    continue
                keep.discard(idx_short)  # short text fully contained
        return keep

    except ImportError:
        # fallback O(n^2) – OK for ≤100k survivors
        texts = {i: df.at[i, col] for i in candidates}
        by_len = sorted(texts.items(), key=lambda kv: len(kv[1]))
        keep: Set[int] = set(texts.keys())
        for j in range(len(by_len)):
            idx_short, short_txt = by_len[j]
            if idx_short not in keep:
                continue
            for idx_long, long_txt in by_len[j + 1 :]:
                if idx_long not in keep:
                    continue
                if short_txt in long_txt:
                    keep.discard(idx_short)
                    break
        return keep
    
def simhash_dedup(
    df: pd.DataFrame,
    candidates: Iterable[int],
    col: str = "segment_norm",
    ngram: int = 4,
    hamming_thresh: int = 3,
) -> Set[int]:
    """Return indices to keep after near‑dup removal via SimHash64."""

    def tok4(text: str) -> List[str]:
        toks = re.sub(r"\W+", " ", text).lower().split()
        if len(toks) < ngram:
            return toks
        return [" ".join(toks[i : i + ngram]) for i in range(len(toks) - ngram + 1)]

    # Build SimhashIndex
    objs = [(str(i), Simhash(tok4(df.at[i, col]), f=64)) for i in candidates]
    index = SimhashIndex(objs, k=hamming_thresh)

    dup_ids: Set[int] = set()
    for doc_id, sim in objs:
        if int(doc_id) in dup_ids:
            continue
        near = index.get_near_dups(sim)
        for d in near:
            if d != doc_id:
                dup_ids.add(int(d))
    keep = {i for i in candidates if i not in dup_ids}
    return keep


In [13]:
def deduplicate(df: pd.DataFrame, col: str = "segment_norm") -> pd.DataFrame:
    """Full three‑stage dedup; returns a *new* DataFrame of unique docs."""

    print("[1/3] MinHash clustering …")
    clusters = minhash_dedup(df, col)
    canon_indices = {
        max(idxs, key=lambda i: len(df.at[i, col])) for idxs in clusters.values()
    }

    print("[2/3] Substring containment filter …")
    canon_indices = containment_dedup(df, canon_indices, col)

    print("[3/3] SimHash fuzzy filter …")
    canon_indices = simhash_dedup(df, canon_indices, col)

    return df.loc[sorted(canon_indices)].reset_index(drop=True)

In [14]:
df_deduped = deduplicate(df_exploded, col="segment_norm")

[1/3] MinHash clustering …


MinHash: build: 0doc [00:00, ?doc/s]

MinHash: query:   0%|          | 0/213364 [00:00<?, ?doc/s]

[2/3] Substring containment filter …
[3/3] SimHash fuzzy filter …


Big bucket found. key:75b:1, len:227
Big bucket found. key:e081:0, len:306
Big bucket found. key:475b:1, len:544
Big bucket found. key:2341:1, len:290
Big bucket found. key:e586:2, len:207
Big bucket found. key:477b:1, len:278
Big bucket found. key:477f:1, len:294
Big bucket found. key:6669:3, len:445
Big bucket found. key:c181:0, len:387
Big bucket found. key:aa65:3, len:424
Big bucket found. key:75b:1, len:227
Big bucket found. key:475b:1, len:544
Big bucket found. key:6c69:3, len:223
Big bucket found. key:77f:1, len:205
Big bucket found. key:475b:1, len:544
Big bucket found. key:6e69:3, len:745
Big bucket found. key:e586:2, len:207
Big bucket found. key:6c69:3, len:223
Big bucket found. key:77f:1, len:205
Big bucket found. key:75b:1, len:227
Big bucket found. key:e586:2, len:207
Big bucket found. key:6c69:3, len:223
Big bucket found. key:475b:1, len:544
Big bucket found. key:ee69:3, len:356
Big bucket found. key:6669:3, len:445
Big bucket found. key:e181:0, len:218
Big bucket found.

In [15]:
df_deduped['segment_word_count'] = df_deduped['segments'].progress_apply(lambda x: factiva_wordcount(x))
df_deduped

  0%|          | 0/146962 [00:00<?, ?it/s]

,index,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,...,region_codes,an,action,company_names,industries,segments,num_segments,segment_norm,segment_hash,segment_word_count
0,0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",...,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",London-based multinational insurance and asset...,5,london-based multinational insurance and asset...,a176a4521fa13765d1ec53bf0557756c,33
1,0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",...,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",\n\n“Subject to obtaining regulatory approvals...,5,"“subject to obtaining regulatory approvals, pr...",d3ebc36206e177294b3af34911353648,157
2,0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",...,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...","\n\nMeanwhile, Bloomberg has reported that bil...",5,"meanwhile, bloomberg has reported that billion...",200f451e94d7320d8bebdde26c5da12d,161
3,0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",...,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",\n\nWith global reinsurance major Swiss RE pro...,5,with global reinsurance major swiss re project...,93fe39a844b1d00b9eeb9a925f165143,59
4,0,The Telegraph,Global insurers pick India ally: Prudential pa...,"'India is a key market for Prudential, and we ...",London-based multinational insurance and asset...,NaN,578,ABPTEL,",i82,i82001,i82002,ifinal,","allanz,bajaal,blfima,hcltec,icplic,paions,pias...",...,",asiaz,devgcoz,dvpcoz,eurz,india,sasiaz,uk,weurz,",ABPTEL0020250322el3m00021,add,"Allianz SE,Bajaj Auto Limited,Bloomberg LP,HCL...","Insurance,Full-line Insurance,Life Insurance,F...",\n\nWith life insurance penetration at a level...,5,with life insurance penetration at a level of ...,c7e0dce171e1e768d3793b239d13bba4,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146957,46675,walesonline.co.uk,Financial services firm that collapsed owing c...,Neal Griffith was chief executive of Optima Wo...,Mr Griffith is the former chief executive and ...,Rugby News,1210,WALESONL,",i836,iacc,ibcs,ifinal,","fisgbo,fsrcsl,moilga,ombud,vhskrn,vibidl",...,",cardi,eurz,uk,wales,weurz,",WALESONL20250407el47004ed,add,"Optima Worldwide Group PLC,Financial Services ...","Accounting,Accounting/Consulting,Business/Cons...",\n\nIn 2016 OWG was admitted to the Copenhagen...,10,in 2016 owg was admitted to the copenhagen-bas...,a63e669288dca1966eda9622b27a36dd,183
146958,46676,walesonline.co.uk,One of Cardiff's biggest employers is moving t...,Car financing firm MotoNovo is moving its HQ f...,"It

## Optional check: Semantic similarity

In [ ]:
#!pip install -U sentence-transformers faiss-gpu-cu12 [works on Linux with NVIDIA GPU, will not work on Mac/Windows, careful!]
# --------------------------------------------------------------------
# 4)  Semantic dedup that handles *long* (≤ 2k-word) segments
# --------------------------------------------------------------------
import math, numpy as np, pandas as pd
import faiss
from tqdm.auto import tqdm
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer

TEXT_COL      = "segments"
MODEL_NAME    = "intfloat/multilingual-e5-large-instruct"
CHUNK_TOKENS  = 350           # window length
STRIDE        = 300           # overlap: CHUNK_TOKENS – STRIDE  (= 50 here)
COS_THRESHOLD = 0.95
BATCH_CHUNKS  = 512           

# ───────────────────  initialise model & tokenizer ───────────────────
model     = SentenceTransformer(MODEL_NAME, device="cuda")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
DIM       = model.get_sentence_embedding_dimension()

# ───────────────────  helper: chunk→embed→mean ───────────────────────
def embed_long(text: str) -> np.ndarray:
    """Mean-pool E5 embeddings of overlapping CHUNK_TOKENS windows."""
    # 1) tokenise once
    ids = tokenizer(text, add_special_tokens=False)["input_ids"]

    # 2) split into overlapping windows
    slices = [
        ids[i : i + CHUNK_TOKENS]
        for i in range(0, len(ids), STRIDE)
    ] or [ids]                          # edge-case: empty → keep one slice

    # 3) decode back to strings for ST encoder
    windows = tokenizer.batch_decode(slices, skip_special_tokens=True)

    # 4) encode windows in smaller sub-batches to fit GPU
    embs, out = [], None
    for s in range(0, len(windows), BATCH_CHUNKS):
        sub = windows[s : s + BATCH_CHUNKS]
        sub_emb = model.encode(
            sub,
            batch_size=len(sub),
            convert_to_numpy=True,
            normalize_embeddings=True,
            show_progress_bar=False,
        )
        embs.append(sub_emb)
    out = np.vstack(embs).mean(axis=0)
    # 5) re-normalise pooled vector → cosine ready
    out /= np.linalg.norm(out) + 1e-12
    return out.astype("float32")

# ────────────────────  encode whole DataFrame  ───────────────────────
texts = df_deduped[TEXT_COL].tolist()
doc_embs = np.empty((len(texts), DIM), dtype="float32")

for i, txt in enumerate(tqdm(texts, desc="Encoding long docs")):
    doc_embs[i] = embed_long(txt)

# ────────────────────  FAISS search & clustering  ────────────────────
index = faiss.IndexFlatIP(DIM)
index.add(doc_embs)

D, I = index.search(doc_embs, 2)  



modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

sentence_xlm-roberta_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Encoding long docs:   0%|          | 0/146962 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1020 > 512). Running this sequence through the model will result in indexing errors


IndexError: list index out of range

In [66]:
N = len(df_deduped)
pairs = []
for i, (sim, j) in tqdm(enumerate(zip(D[:, 1], I[:, 1]))):
    j = int(j)
    if 0 <= j < N and i != j and sim >= COS_THRESHOLD:
        pairs.append((i, j))

parent = list(range(len(df_deduped)))
size   = [1]*len(df_deduped)

def find(x):
    while parent[x] != x:
        parent[x] = parent[parent[x]]
        x = parent[x]
    return x

def union(a, b):
    ra, rb = find(a), find(b)
    if ra == rb:
        return
    if size[ra] < size[rb]:
        ra, rb = rb, ra
    parent[rb] = ra
    size[ra]  += size[rb]

for a, b in tqdm(pairs):
    union(a, b)
    
clusters = {}
for idx in tqdm(range(len(df_deduped))):
    clusters.setdefault(find(idx), []).append(idx)

keep_idx = {
    max(ids, key=lambda i: len(df_deduped.at[i, TEXT_COL]))
    for ids in clusters.values()
}

df_semantic = df_deduped.loc[sorted(keep_idx)].reset_index(drop=True)
print(f"Semantic dedup kept {len(df_semantic)} / {len(df_deduped)} rows")

0it [00:00, ?it/s]

  0%|          | 0/99082 [00:00<?, ?it/s]

  0%|          | 0/146962 [00:00<?, ?it/s]

Semantic dedup kept 76974 / 146962 rows


## Exporting results

In [68]:
df_semantic.to_csv(f'../../extractions/{job_id}/result_deduped_semantic.csv', index=False)

In [ ]:
# df.to_csv('../../extractions/dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-l0fy7lkzhf/preprocessing_result.csv', index=False, encoding='utf-8')

In [16]:
df_deduped.rename(columns={'index':'original_index'}, inplace=True)
df_deduped.to_csv(f'../../extractions/{job_id}/preprocessing_result_deduped.csv', index=False, encoding='utf-8')